# Experiment 1: Retrieval Strategy Baseline Comparison

**Analysis & Visualization for Thesis**

This notebook provides comprehensive analysis and publication-ready visualizations for Experiment 1, which compares five retrieval strategies:

1. **BM25 Baseline** - Traditional sparse retrieval
2. **SPLADE Baseline** - Neural sparse retrieval
3. **Dense BGE-M3** - Dense semantic retrieval
4. **Hybrid SPLADE + BGE-M3** - Hybrid approach
5. **Hybrid BM25 + BGE-M3** - Traditional hybrid

**Dataset:** StackOverflow (SOSum) - 506 queries

**Date:** October 5, 2025

## Πειραματική Διαδικασία (Experimental Setup)

### Δημιουργία Ground Truth

Για την αξιολόγηση των συστημάτων ανάκτησης πληροφορίας, δημιουργήθηκε ground truth σε επίπεδο **chunks** (τμημάτων εγγράφων), όχι σε επίπεδο ολόκληρων εγγράφων. Συγκεκριμένα:

- **Corpus:** Το σύνολο δεδομένων SOSum (Stack Overflow) περιέχει 506 ερωτήματα (queries)
- **Chunking Strategy:** Κάθε έγγραφο (Stack Overflow answer) χωρίζεται σε chunks μεγέθους 512 tokens με overlap 50 tokens
- **Ground Truth Mapping:** Για κάθε query, τα relevant chunks προσδιορίζονται με βάση το αρχικό relevant document. Δηλαδή, αν ένα document D είναι relevant για το query Q, τότε **όλα τα chunks** που προέρχονται από το D θεωρούνται relevant για το Q.

### Αξιολόγηση σε Επίπεδο Chunks

Η αξιολόγηση γίνεται σε επίπεδο chunks, όχι documents:

- **Retrieval Results:** Κάθε σύστημα επιστρέφει τα top-k chunks για κάθε query
- **Relevance Judgment:** Ένα retrieved chunk θεωρείται σωστό (relevant) αν προέρχεται από ένα document που είναι relevant για το συγκεκριμένο query
- **Metrics Calculation:** Οι μετρικές (Precision, Recall, F1, MAP, MRR, NDCG) υπολογίζονται με βάση το πόσα από τα retrieved chunks είναι relevant chunks

### Στόχοι Experiment 1: Baseline Comparison

Το **Experiment 1** έχει ως στόχο τη **σύγκριση πέντε βασικών στρατηγικών ανάκτησης** (retrieval strategies) χωρίς την εφαρμογή reranking:

1. **BM25 Baseline** - Παραδοσιακή λεξικογραφική (sparse) μέθοδος
2. **SPLADE Baseline** - Νευρωνική sparse μέθοδος με learned term expansion
3. **Dense BGE-M3** - Πυκνή σημασιολογική (dense semantic) ανάκτηση
4. **Hybrid SPLADE + BGE-M3** - Υβριδική προσέγγιση (0.5 sparse + 0.5 dense)
5. **Hybrid BM25 + BGE-M3** - Παραδοσιακή υβριδική προσέγγιση (0.5 BM25 + 0.5 dense)

**Παρατηρήσεις:**
- Η αξιολόγηση εστιάζει κυρίως σε **set-based metrics** (Precision, Recall, F1, MAP) λόγω του noise στο ranking του Stack Overflow
- Οι rank-aware μετρικές (MRR, NDCG) παρουσιάζονται για σχετική σύγκριση μεταξύ των συστημάτων
- Τα αποτελέσματα χρησιμοποιούνται για την επιλογή των καλύτερων retrieval strategies για το **Experiment 2** (reranking optimization)

## 1. Setup & Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Ρυθμίσεις matplotlib για ελληνικά (matching llm_judge_plots)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['DejaVu Serif', 'Times New Roman', 'Liberation Serif']
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['axes.unicode_minus'] = False

# Larger font sizes for report readability
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 16

# IBM Carbon color palette for the 5 retrieval methods
ibm_colors = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=ibm_colors)

print("✓ Βιβλιοθήκες φορτώθηκαν επιτυχώς")

In [ ]:
# Define paths
RESULTS_DIR = Path('../../results/experiment_1')
OUTPUT_DIR = Path('../../output/experiment_1_plots')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load data files (corrected file patterns)
summary_file = list(RESULTS_DIR.glob('experiment_summary_full_*.csv'))[-1]
stats_file = list(RESULTS_DIR.glob('experiment_statistical_analysis_full_*.csv'))[-1]
full_results_file = list(RESULTS_DIR.glob('experiment_full_results_full_*.json'))[-1]

print(f"Loading data from:")
print(f"  Summary: {summary_file.name}")
print(f"  Statistics: {stats_file.name}")
print(f"  Full Results: {full_results_file.name}")

# Load CSV files
df_summary = pd.read_csv(summary_file)
df_stats = pd.read_csv(stats_file)

# Load JSON file
with open(full_results_file, 'r') as f:
    full_results = json.load(f)

print("\n✅ Data loaded successfully")
print(f"\nDataset info:")
print(f"  Scenarios: {len(df_summary)}")
print(f"  Statistical comparisons: {len(df_stats)}")
print(f"  Total queries per scenario: {df_summary['total_queries'].iloc[0]}")

In [ ]:
# Display summary statistics (including k=10 for Experiment 2 optimization)
print("\n📊 Summary Table\n")
display_cols = ['scenario', 'retrieval_type', 'precision@10_mean', 'recall@10_mean', 
                'f1@10_mean', 'map_mean', 'mrr_mean', 'ndcg@10_mean', 'time_mean_ms']
df_summary[display_cols].round(4)

## 2. Data Preparation & Color Scheme

In [ ]:
# IBM Carbon Categorical Color Palette (Light theme)
# Curated sequence for maximum contrast between neighboring colors
CARBON_COLORS = {
    'purple_70': '#6929c4',   # 01. Purple 70
    'cyan_50': '#1192e8',     # 02. Cyan 50
    'teal_70': '#005d5d',     # 03. Teal 70
    'magenta_70': '#9f1853',  # 04. Magenta 70
    'red_50': '#fa4d56',      # 05. Red 50
    'red_90': '#570408',      # 06. Red 90
    'green_60': '#198038',    # 07. Green 60
    'blue_80': '#002d9c',     # 08. Blue 80
    'magenta_50': '#ee538b',  # 09. Magenta 50
    'yellow_50': '#b28600',   # 10. Yellow 50
    'teal_50': '#009d9a',     # 11. Teal 50
    'cyan_90': '#012749',     # 12. Cyan 90
    'orange_70': '#8a3800',   # 13. Orange 70
    'purple_50': '#a56eff'    # 14. Purple 50
}

# Apply Carbon colors to retrieval methods (5 distinct methods)
# Using sequence order for maximum contrast between neighbors
COLOR_SCHEME = {
    'BM25_Baseline': CARBON_COLORS['purple_70'],        # 01. Purple - Traditional baseline
    'SPLADE_Baseline': CARBON_COLORS['cyan_50'],        # 02. Cyan - Neural sparse  
    'Dense_BGE_M3': CARBON_COLORS['green_60'],          # 07. Green - Dense semantic (skipped for contrast)
    'Hybrid_SPLADE_BGE_M3': CARBON_COLORS['magenta_70'], # 04. Magenta - Best hybrid
    'Hybrid_BM25_BGE_M3': CARBON_COLORS['yellow_50']    # 10. Yellow - Traditional hybrid
}

# For reference - named colors
colors_thesis = {
    'primary': CARBON_COLORS['cyan_50'],
    'secondary': CARBON_COLORS['purple_70'],
    'accent': CARBON_COLORS['magenta_70'],
    'success': CARBON_COLORS['green_60'],
    'warning': CARBON_COLORS['yellow_50'],
    'danger': CARBON_COLORS['red_50'],
    'grid': '#e0e0e0',       # Light gray for grid
    'text': '#161616'        # Carbon text color
}

# Readable labels for plots
LABEL_MAPPING = {
    'BM25_Baseline': 'BM25',
    'SPLADE_Baseline': 'SPLADE',
    'Dense_BGE_M3': 'Dense\n(BGE-M3)',
    'Hybrid_SPLADE_BGE_M3': 'Hybrid\n(SPLADE+BGE)',
    'Hybrid_BM25_BGE_M3': 'Hybrid\n(BM25+BGE)'
}

# Short labels for compact plots
SHORT_LABELS = {
    'BM25_Baseline': 'BM25',
    'SPLADE_Baseline': 'SPLADE',
    'Dense_BGE_M3': 'Dense',
    'Hybrid_SPLADE_BGE_M3': 'Hybrid-S',
    'Hybrid_BM25_BGE_M3': 'Hybrid-B'
}

# Order for plots
SCENARIO_ORDER = ['BM25_Baseline', 'SPLADE_Baseline', 'Dense_BGE_M3', 
                  'Hybrid_SPLADE_BGE_M3', 'Hybrid_BM25_BGE_M3']

print("✅ IBM Carbon color palette applied (categorical - maximized contrast)")
print("   Colors: Purple → Cyan → Green → Magenta → Yellow")

## 3. Visualization Functions

In [ ]:
def save_figure(fig, filename, formats=['png', 'pdf']):
    """Save figure in multiple formats for thesis."""
    for fmt in formats:
        filepath = OUTPUT_DIR / f"{filename}.{fmt}"
        fig.savefig(filepath, format=fmt, bbox_inches='tight', dpi=300)
        print(f"  ✓ Saved: {filepath}")

def calculate_ci_95(mean, std, n=506):
    """Calculate 95% confidence interval (±1.96 * SE)."""
    se = std / np.sqrt(n)
    ci = 1.96 * se
    return ci

def format_percentage(value, decimals=1):
    """Format value as percentage."""
    return f"{value*100:.{decimals}f}%"

print("✅ Helper functions defined")

## 4. Metric Interpretation Strategy

### Understanding Dataset Limitations

The **SOSum dataset** uses Stack Overflow's default ranking system (community voting + accepted answers), which doesn't always represent perfect semantic ordering. This affects **rank-aware metrics** (NDCG, MRR) that assume the first answer is most relevant.

**Our Approach:**
1. **Primary Focus**: Set-based metrics (Precision@k, Recall@k, F1@k, MAP) - less sensitive to ranking noise
2. **Secondary Analysis**: Rank-aware metrics (NDCG, MRR) - valuable for *relative comparison* between methods
3. **Confidence Intervals**: Use 95% CI instead of raw standard deviation for cleaner visualization

**Rationale**: All retrieval methods face the same noisy ground truth, so relative comparisons remain valid. Real-world data with inherent imperfections is preferable to artificial benchmarks.

**Note**: Error bars in all plots show 95% confidence intervals (±1.96 × SE), not standard deviation.

## 5. Figure 1: Overall Performance Comparison (MAP, MRR, NDCG@10)

In [ ]:
# Prepare data for overall performance metrics
metrics_to_plot = ['map_mean', 'mrr_mean', 'ndcg@10_mean']
metric_labels = ['MAP', 'MRR', 'NDCG@10']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # Larger size
fig.suptitle('Συνολική επίδοση του συστήματος ανάκτησης', fontsize=16, fontweight='bold', y=1.02)

for idx, (metric, label) in enumerate(zip(metrics_to_plot, metric_labels)):
    ax = axes[idx]
    
    # Extract data
    data = df_summary.set_index('scenario').loc[SCENARIO_ORDER]
    values = data[metric].values
    std_col = metric.replace('_mean', '_std')
    stds = data[std_col].values
    # Use 95% CI for cleaner visualization
    errors = [calculate_ci_95(v, s) for v, s in zip(values, stds)]
    
    # Create bar plot
    bars = ax.bar(range(len(SCENARIO_ORDER)), values, 
                   color=[COLOR_SCHEME[s] for s in SCENARIO_ORDER],
                   yerr=errors, capsize=5, alpha=0.85, edgecolor='black', 
                   linewidth=1.0, error_kw={'linewidth': 1.5, 'alpha': 0.7})
    
    # Customize
    ax.set_ylabel(label, fontweight='bold', fontsize=14)
    ax.set_xticks(range(len(SCENARIO_ORDER)))
    ax.set_xticklabels([SHORT_LABELS[s] for s in SCENARIO_ORDER], rotation=45, ha='right', fontsize=12)
    ax.set_ylim(0, max(values) * 1.2)
    ax.grid(axis='y', alpha=0.3, linestyle=':')
    
    # Add value labels on bars
    for i, (bar, val) in enumerate(zip(bars, values)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + errors[i] + 0.01,
                f'{val:.3f}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Highlight best performer
    best_idx = np.argmax(values)
    bars[best_idx].set_edgecolor('gold')
    bars[best_idx].set_linewidth(3)

plt.tight_layout()
save_figure(fig, 'fig1_overall_performance')
plt.show()

print("\n✅ Figure 1 generated: Overall Performance Comparison")
print("   Note: Error bars show 95% confidence intervals (not std)")

## 6. Figure 2: Precision@k Comparison

In [ ]:
# Precision at different k values (including k=10 for Experiment 2)
k_values = [1, 3, 5, 10]
precision_metrics = [f'precision@{k}_mean' for k in k_values]

fig, ax = plt.subplots(figsize=(14, 7))  # Larger size

x = np.arange(len(k_values))
width = 0.15

for i, scenario in enumerate(SCENARIO_ORDER):
    data = df_summary[df_summary['scenario'] == scenario]
    values = [data[metric].values[0] for metric in precision_metrics]
    # Use 95% CI instead of std for cleaner visualization
    stds = [data[metric.replace('_mean', '_std')].values[0] for metric in precision_metrics]
    errors = [calculate_ci_95(v, s) for v, s in zip(values, stds)]
    
    offset = (i - len(SCENARIO_ORDER)/2 + 0.5) * width
    bars = ax.bar(x + offset, values, width, label=SHORT_LABELS[scenario],
                   color=COLOR_SCHEME[scenario], alpha=0.85, 
                   yerr=errors, capsize=3, edgecolor='black', linewidth=0.8, 
                   error_kw={'linewidth': 1.5, 'alpha': 0.7})

ax.set_xlabel('k (πλήθος ανακτημένων τμημάτων)', fontweight='bold', fontsize=14)
ax.set_ylabel('Precision@k', fontweight='bold', fontsize=14)
ax.set_title('Ακρίβεια για διαφορετικές τιμές αποκοπής', 
             fontsize=15, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels([f'k={k}' for k in k_values], fontsize=12)
ax.legend(loc='upper right', framealpha=0.95, edgecolor='gray', fontsize=12)
ax.set_ylim(0, 1.0)
ax.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
save_figure(fig, 'fig2_precision_at_k')
plt.show()

print("\n✅ Figure 2 generated: Precision@k Comparison (with k=10)")

## 6. Figure 3: Recall@k Comparison

In [ ]:
# Recall at different k values (including k=10)
recall_metrics = [f'recall@{k}_mean' for k in k_values]

fig, ax = plt.subplots(figsize=(14, 7))  # Larger size

x = np.arange(len(k_values))
width = 0.15

for i, scenario in enumerate(SCENARIO_ORDER):
    data = df_summary[df_summary['scenario'] == scenario]
    values = [data[metric].values[0] for metric in recall_metrics]
    # Use 95% CI instead of std
    stds = [data[metric.replace('_mean', '_std')].values[0] for metric in recall_metrics]
    errors = [calculate_ci_95(v, s) for v, s in zip(values, stds)]
    
    offset = (i - len(SCENARIO_ORDER)/2 + 0.5) * width
    bars = ax.bar(x + offset, values, width, label=SHORT_LABELS[scenario],
                   color=COLOR_SCHEME[scenario], alpha=0.85,
                   yerr=errors, capsize=3, edgecolor='black', linewidth=0.8,
                   error_kw={'linewidth': 1.5, 'alpha': 0.7})

ax.set_xlabel('k (πλήθος ανακτημένων τμημάτων)', fontweight='bold', fontsize=14)
ax.set_ylabel('Recall@k', fontweight='bold', fontsize=14)
ax.set_title('Ανάκληση για διαφορετικές τιμές αποκοπής', 
             fontsize=15, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels([f'k={k}' for k in k_values], fontsize=12)
ax.legend(loc='upper left', framealpha=0.95, edgecolor='gray', fontsize=12)
ax.set_ylim(0, max([df_summary[metric].max() for metric in recall_metrics]) * 1.15)
ax.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
save_figure(fig, 'fig3_recall_at_k')
plt.show()

print("\n✅ Figure 3 generated: Recall@k Comparison (with k=10)")

## 7. Figure 4: F1@k Scores

In [ ]:
# F1 scores at different k values (including k=10 for consistency)
f1_k_values = [3, 5, 10]
f1_metrics = [f'f1@{k}_mean' for k in f1_k_values]

fig, ax = plt.subplots(figsize=(14, 7))  # Larger size

x = np.arange(len(f1_k_values))
width = 0.15

for i, scenario in enumerate(SCENARIO_ORDER):
    data = df_summary[df_summary['scenario'] == scenario]
    values = [data[metric].values[0] for metric in f1_metrics]
    stds = [data[metric.replace('_mean', '_std')].values[0] for metric in f1_metrics]
    errors = [calculate_ci_95(v, s) for v, s in zip(values, stds)]
    
    offset = (i - len(SCENARIO_ORDER)/2 + 0.5) * width
    bars = ax.bar(x + offset, values, width, label=SHORT_LABELS[scenario],
                   color=COLOR_SCHEME[scenario], alpha=0.85,
                   yerr=errors, capsize=3, edgecolor='black', linewidth=0.8,
                   error_kw={'linewidth': 1.5, 'alpha': 0.7})

ax.set_xlabel('k (πλήθος ανακτημένων τμημάτων)', fontweight='bold', fontsize=14)
ax.set_ylabel('F1@k', fontweight='bold', fontsize=14)
ax.set_title('F1 για διαφορετικές τιμές αποκοπής', 
             fontsize=15, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels([f'k={k}' for k in f1_k_values], fontsize=12)
ax.legend(loc='lower right', framealpha=0.95, edgecolor='gray', fontsize=12)
ax.set_ylim(0, max([df_summary[metric].max() for metric in f1_metrics]) * 1.15)
ax.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
save_figure(fig, 'fig4_f1_scores')
plt.show()

print("\n✅ Figure 4 generated: F1@k Scores (with k=10)")

## 8. Figure 5: Precision-Recall Tradeoff

In [ ]:
# Precision-Recall scatter plot at k=5
fig, ax = plt.subplots(figsize=(12, 10))  # Larger size

for scenario in SCENARIO_ORDER:
    data = df_summary[df_summary['scenario'] == scenario]
    precision = data['precision@5_mean'].values[0]
    recall = data['recall@5_mean'].values[0]
    p_std = data['precision@5_std'].values[0]
    r_std = data['recall@5_std'].values[0]
    
    # Use 95% CI for error bars
    p_ci = calculate_ci_95(precision, p_std)
    r_ci = calculate_ci_95(recall, r_std)
    
    # Plot point with error bars
    ax.errorbar(recall, precision, xerr=r_ci, yerr=p_ci,
                marker='o', markersize=16, label=SHORT_LABELS[scenario],
                color=COLOR_SCHEME[scenario], capsize=6, capthick=2.5,
                linewidth=3, alpha=0.8)

# Add quadrant lines
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.4, linewidth=1.5)
ax.axvline(x=0.3, color='gray', linestyle='--', alpha=0.4, linewidth=1.5)

ax.set_xlabel('Recall@5', fontweight='bold', fontsize=14)
ax.set_ylabel('Precision@5', fontweight='bold', fontsize=14)
ax.set_title('Συμβιβασμός ακρίβειας-ανάκλησης για k = 5\n', 
             fontsize=15, fontweight='bold', pad=15)
ax.legend(loc='lower left', framealpha=0.95, edgecolor='gray', fontsize=12)
ax.grid(True, alpha=0.3, linestyle=':')
ax.set_xlim(0, max(df_summary['recall@5_mean']) * 1.15)
ax.set_ylim(0, 1.0)

# Add text annotation for ideal quadrant
ax.text(0.45, 0.95, 'Υψηλή ακρίβεια\nΥψηλή ανάκληση', ha='center', fontsize=11, 
        style='italic', alpha=0.6, fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='wheat', alpha=0.3, edgecolor='gray'))

plt.tight_layout()
save_figure(fig, 'fig5_precision_recall_tradeoff')
plt.show()

print("\n✅ Figure 5 generated: Precision-Recall Tradeoff")

## 9. Figure 6: NDCG@k Progression

In [ ]:
# NDCG at different cutoffs (including k=10)
ndcg_k_values = [1, 3, 5, 10]
ndcg_metrics = [f'ndcg@{k}_mean' for k in ndcg_k_values]

fig, ax = plt.subplots(figsize=(14, 7))  # Larger size

for scenario in SCENARIO_ORDER:
    data = df_summary[df_summary['scenario'] == scenario]
    values = [data[metric].values[0] for metric in ndcg_metrics]
    
    ax.plot(ndcg_k_values, values, marker='o', markersize=12, 
            linewidth=3.5, label=SHORT_LABELS[scenario],
            color=COLOR_SCHEME[scenario], alpha=0.85)

ax.set_xlabel('k', fontweight='bold', fontsize=14)
ax.set_ylabel('NDCG@k', fontweight='bold', fontsize=14)
ax.set_title('Normalized Discounted Cumulative Gain για διαφορετικές τιμές αποκοπής', 
             fontsize=15, fontweight='bold', pad=15)
ax.set_xticks(ndcg_k_values)
ax.set_xticklabels([f'k={k}' for k in ndcg_k_values], fontsize=12)
ax.legend(loc='lower left', framealpha=0.95, edgecolor='gray', fontsize=12)
ax.set_ylim(0, 1.0)
ax.grid(True, alpha=0.3, linestyle=':')

plt.tight_layout()
save_figure(fig, 'fig6_ndcg_progression')
plt.show()

print("\n✅ Figure 6 generated: NDCG@k Progression (with k=10)")

## 10. Figure 7: Query Latency Analysis

In [ ]:
# Latency comparison (without error bars due to high local variance)
fig, axes = plt.subplots(1, 2, figsize=(16, 7))  # Larger size

# Plot 1: Mean latency (no error bars to avoid obscuring)
ax = axes[0]
data = df_summary.set_index('scenario').loc[SCENARIO_ORDER]
means = data['time_mean_ms'].values

bars = ax.barh(range(len(SCENARIO_ORDER)), means,
               color=[COLOR_SCHEME[s] for s in SCENARIO_ORDER],
               alpha=0.85, edgecolor='black', linewidth=1.2)

ax.set_yticks(range(len(SCENARIO_ORDER)))
ax.set_yticklabels([SHORT_LABELS[s] for s in SCENARIO_ORDER], fontsize=12)
ax.set_xlabel('Μέση Καθυστέρηση (ms)', fontweight='bold', fontsize=14)
ax.set_title('Σύγκριση Καθυστέρησης Ερωτημάτων\n(Τοπική εκτέλεση - υψηλή διασπορά)', 
             fontweight='bold', fontsize=15)
ax.grid(axis='x', alpha=0.3, linestyle=':')

# Add value labels
for i, (bar, mean) in enumerate(zip(bars, means)):
    ax.text(mean + 20, bar.get_y() + bar.get_height()/2,
            f'{mean:.1f} ms',
            va='center', ha='left', fontsize=11, fontweight='bold')

# Plot 2: Latency percentiles (cleaner visualization)
ax = axes[1]
metrics_latency = ['time_median_ms', 'time_p95_ms']
x = np.arange(len(SCENARIO_ORDER))
width = 0.35

median_vals = data['time_median_ms'].values
p95_vals = data['time_p95_ms'].values

bars1 = ax.bar(x - width/2, median_vals, width, label='Διάμεσος (P50)',
               color=COLORS[0], alpha=0.85, 
               edgecolor='black', linewidth=1)
bars2 = ax.bar(x + width/2, p95_vals, width, label='P95',
               color=COLORS[3], alpha=0.85, 
               edgecolor='black', linewidth=1)

ax.set_ylabel('Καθυστέρηση (ms)', fontweight='bold', fontsize=14)
ax.set_title('Εκατοστημόρια Καθυστέρησης', fontweight='bold', fontsize=15)
ax.set_xticks(x)
ax.set_xticklabels([SHORT_LABELS[s] for s in SCENARIO_ORDER], 
                   rotation=45, ha='right', fontsize=11)
ax.legend(framealpha=0.95, edgecolor='gray', fontsize=12)
ax.grid(axis='y', alpha=0.3, linestyle=':')

plt.tight_layout()
save_figure(fig, 'fig7_latency_analysis')
plt.show()

print("\n✅ Figure 7 generated: Query Latency Analysis (cleaned up)")

## 11. Figure 8: Statistical Significance Heatmap

In [ ]:
# Create pairwise comparison matrix for MAP metric
map_stats = df_stats[df_stats['metric'] == 'map'].copy()

# Create matrix
scenarios = SCENARIO_ORDER
n = len(scenarios)
significance_matrix = np.zeros((n, n))
effect_size_matrix = np.zeros((n, n))

for _, row in map_stats.iterrows():
    if row['method1'] in scenarios and row['method2'] in scenarios:
        i = scenarios.index(row['method1'])
        j = scenarios.index(row['method2'])
        
        # Significance (1 = significant, 0 = not significant)
        significance_matrix[i, j] = 1 if row['bonferroni_significant'] else 0
        significance_matrix[j, i] = 1 if row['bonferroni_significant'] else 0  # Symmetric
        
        # Effect size (symmetric but with opposite sign)
        if pd.notna(row['effect_size']):
            effect_size_matrix[i, j] = row['effect_size']
            effect_size_matrix[j, i] = -row['effect_size']  # Opposite sign for symmetric position

# Create improved heatmap with better styling
fig, axes = plt.subplots(1, 2, figsize=(20, 9))  # Larger size
fig.suptitle('Statistical Significance Analysis - MAP Metric', 
             fontsize=18, fontweight='bold', y=0.98)

# --- LEFT: Significance heatmap with cleaner design ---
ax = axes[0]

# Use discrete colormap for binary significance
from matplotlib.colors import ListedColormap
sig_colors = ['#fee5d9', '#a50f15']  # Light red (non-sig) to dark red (sig)
sig_cmap = ListedColormap(sig_colors)

im1 = ax.imshow(significance_matrix, cmap=sig_cmap, aspect='auto', vmin=0, vmax=1, 
                interpolation='nearest', alpha=0.9)

# Add gridlines for clarity
for i in range(n+1):
    ax.axhline(i-0.5, color='white', linewidth=2.5)
    ax.axvline(i-0.5, color='white', linewidth=2.5)

ax.set_xticks(np.arange(n))
ax.set_yticks(np.arange(n))
ax.set_xticklabels([SHORT_LABELS[s] for s in scenarios], rotation=45, ha='right', fontsize=13)
ax.set_yticklabels([SHORT_LABELS[s] for s in scenarios], fontsize=13)
ax.set_title('Statistical Significance\n(Bonferroni Corrected p < 0.05)', 
             fontweight='bold', pad=20, fontsize=15)

# Add clean annotations with stars for significance
for i in range(n):
    for j in range(n):
        if i == j:
            # Diagonal - same method
            ax.add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1, 
                                       fill=True, facecolor='lightgray', 
                                       edgecolor='white', linewidth=2.5, alpha=0.5))
            ax.text(j, i, '—', ha="center", va="center", 
                   color='gray', fontsize=20, fontweight='bold')
        else:
            # Off-diagonal - show significance
            if significance_matrix[i, j] == 1:
                ax.text(j, i, '***', ha="center", va="center", 
                       color='white', fontsize=24, fontweight='bold')
            else:
                ax.text(j, i, 'n.s.', ha="center", va="center", 
                       color='#666', fontsize=11, style='italic', alpha=0.7)

# Custom colorbar
cbar1 = plt.colorbar(im1, ax=ax, ticks=[0.25, 0.75], pad=0.02)
cbar1.ax.set_yticklabels(['Not Significant', 'Significant'], fontsize=11)
cbar1.ax.tick_params(size=0)

# --- RIGHT: Effect size heatmap with improved styling ---
ax = axes[1]

# Use diverging colormap centered at 0
from matplotlib.colors import TwoSlopeNorm
norm = TwoSlopeNorm(vmin=-1.5, vcenter=0, vmax=1.5)
im2 = ax.imshow(effect_size_matrix, cmap='RdBu_r', aspect='auto', norm=norm, 
                interpolation='nearest', alpha=0.9)

# Add gridlines
for i in range(n+1):
    ax.axhline(i-0.5, color='white', linewidth=2.5)
    ax.axvline(i-0.5, color='white', linewidth=2.5)

ax.set_xticks(np.arange(n))
ax.set_yticks(np.arange(n))
ax.set_xticklabels([SHORT_LABELS[s] for s in scenarios], rotation=45, ha='right', fontsize=13)
ax.set_yticklabels([SHORT_LABELS[s] for s in scenarios], fontsize=13)
ax.set_title('Effect Size (Cohen\'s d)\nPositive = Row > Column', 
             fontweight='bold', pad=20, fontsize=15)

# Add clean annotations with better visibility
for i in range(n):
    for j in range(n):
        if i == j:
            # Diagonal
            ax.add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1, 
                                       fill=True, facecolor='lightgray', 
                                       edgecolor='white', linewidth=2.5, alpha=0.5))
            ax.text(j, i, '—', ha="center", va="center", 
                   color='gray', fontsize=20, fontweight='bold')
        elif effect_size_matrix[i, j] != 0:
            # Show effect size with color-adaptive text
            value = effect_size_matrix[i, j]
            # Choose text color based on background intensity
            text_color = 'white' if abs(value) > 0.5 else 'black'
            
            ax.text(j, i, f'{value:.2f}', ha="center", va="center", 
                   color=text_color, fontsize=13, fontweight='bold')
            
            # Add effect size interpretation as subscript
            if abs(value) >= 0.8:
                size_label = 'L'  # Large
            elif abs(value) >= 0.5:
                size_label = 'M'  # Medium
            elif abs(value) >= 0.2:
                size_label = 'S'  # Small
            else:
                size_label = ''
            
            if size_label:
                ax.text(j, i+0.35, size_label, ha="center", va="center", 
                       color=text_color, fontsize=9, style='italic', alpha=0.8)

# Custom colorbar with interpretations
cbar2 = plt.colorbar(im2, ax=ax, pad=0.02)
cbar2.set_label('Effect Size', fontsize=12, fontweight='bold')
cbar2.ax.tick_params(labelsize=11)

# Add effect size interpretation legend
legend_text = 'S: Small (0.2)\nM: Medium (0.5)\nL: Large (0.8)'
ax.text(1.25, -0.15, legend_text, transform=ax.transAxes,
        fontsize=10, verticalalignment='top', style='italic',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='wheat', alpha=0.3, edgecolor='gray'))

plt.tight_layout(rect=[0, 0, 1, 0.96])
save_figure(fig, 'fig8_statistical_significance')
plt.show()

print("\n✅ Figure 8 generated: Statistical Significance Analysis (improved design)")
print("   - Stars (***) indicate statistical significance (p < 0.05)")
print("   - Effect sizes labeled with magnitude (S/M/L)")
print("   - n.s. = not significant")

## 12. Figure 9: Comprehensive Summary Dashboard

In [ ]:
# Create comprehensive summary figure
fig = plt.figure(figsize=(22, 12))  # Much larger size
gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.35)

# 1. MAP comparison
ax1 = fig.add_subplot(gs[0, 0])
data = df_summary.set_index('scenario').loc[SCENARIO_ORDER]
bars = ax1.bar(range(len(SCENARIO_ORDER)), data['map_mean'].values,
               color=[COLOR_SCHEME[s] for s in SCENARIO_ORDER], alpha=0.85,
               edgecolor='black', linewidth=1.2)
ax1.set_title('MAP', fontweight='bold', fontsize=14)
ax1.set_xticks(range(len(SCENARIO_ORDER)))
ax1.set_xticklabels([SHORT_LABELS[s] for s in SCENARIO_ORDER], rotation=45, ha='right', fontsize=11)
ax1.grid(axis='y', alpha=0.3, linestyle=':')
ax1.tick_params(axis='y', labelsize=11)
# Highlight best
best_idx = np.argmax(data['map_mean'].values)
bars[best_idx].set_edgecolor('gold')
bars[best_idx].set_linewidth(3.5)

# 2. MRR comparison
ax2 = fig.add_subplot(gs[0, 1])
bars = ax2.bar(range(len(SCENARIO_ORDER)), data['mrr_mean'].values,
               color=[COLOR_SCHEME[s] for s in SCENARIO_ORDER], alpha=0.85,
               edgecolor='black', linewidth=1.2)
ax2.set_title('MRR*', fontweight='bold', fontsize=14)
ax2.set_xticks(range(len(SCENARIO_ORDER)))
ax2.set_xticklabels([SHORT_LABELS[s] for s in SCENARIO_ORDER], rotation=45, ha='right', fontsize=11)
ax2.grid(axis='y', alpha=0.3, linestyle=':')
ax2.tick_params(axis='y', labelsize=11)
best_idx = np.argmax(data['mrr_mean'].values)
bars[best_idx].set_edgecolor('gold')
bars[best_idx].set_linewidth(3.5)

# 3. NDCG@10 comparison
ax3 = fig.add_subplot(gs[0, 2])
bars = ax3.bar(range(len(SCENARIO_ORDER)), data['ndcg@10_mean'].values,
               color=[COLOR_SCHEME[s] for s in SCENARIO_ORDER], alpha=0.85,
               edgecolor='black', linewidth=1.2)
ax3.set_title('NDCG@10*', fontweight='bold', fontsize=14)
ax3.set_xticks(range(len(SCENARIO_ORDER)))
ax3.set_xticklabels([SHORT_LABELS[s] for s in SCENARIO_ORDER], rotation=45, ha='right', fontsize=11)
ax3.grid(axis='y', alpha=0.3, linestyle=':')
ax3.tick_params(axis='y', labelsize=11)
best_idx = np.argmax(data['ndcg@10_mean'].values)
bars[best_idx].set_edgecolor('gold')
bars[best_idx].set_linewidth(3.5)

# 4. Precision@k curves (with k=10)
ax4 = fig.add_subplot(gs[1, :])
k_vals = [1, 3, 5, 10]
for scenario in SCENARIO_ORDER:
    scenario_data = df_summary[df_summary['scenario'] == scenario]
    values = [scenario_data[f'precision@{k}_mean'].values[0] for k in k_vals]
    ax4.plot(k_vals, values, marker='o', markersize=13, 
            linewidth=3.5, label=SHORT_LABELS[scenario],
            color=COLOR_SCHEME[scenario], alpha=0.85)
ax4.set_xlabel('k', fontweight='bold', fontsize=14)
ax4.set_ylabel('Precision@k', fontweight='bold', fontsize=14)
ax4.set_title('Precision at Different Cutoffs', fontweight='bold', fontsize=15)
ax4.legend(loc='upper right', ncol=5, framealpha=0.95, fontsize=12, edgecolor='gray')
ax4.grid(True, alpha=0.3, linestyle=':')
ax4.set_xticks(k_vals)
ax4.tick_params(axis='both', labelsize=12)

# 5. Recall@k curves (with k=10)
ax5 = fig.add_subplot(gs[2, :])
for scenario in SCENARIO_ORDER:
    scenario_data = df_summary[df_summary['scenario'] == scenario]
    values = [scenario_data[f'recall@{k}_mean'].values[0] for k in k_vals]
    ax5.plot(k_vals, values, marker='s', markersize=13, 
            linewidth=3.5, label=SHORT_LABELS[scenario],
            color=COLOR_SCHEME[scenario], alpha=0.85)
ax5.set_xlabel('k', fontweight='bold', fontsize=14)
ax5.set_ylabel('Recall@k', fontweight='bold', fontsize=14)
ax5.set_title('Recall at Different Cutoffs', fontweight='bold', fontsize=15)
ax5.legend(loc='lower right', ncol=5, framealpha=0.95, fontsize=12, edgecolor='gray')
ax5.grid(True, alpha=0.3, linestyle=':')
ax5.set_xticks(k_vals)
ax5.tick_params(axis='both', labelsize=12)

fig.suptitle('Experiment 1: Retrieval Strategy Comprehensive Comparison\n(*Rank-aware metrics)', 
             fontsize=18, fontweight='bold', y=0.995)

save_figure(fig, 'fig9_comprehensive_dashboard')
plt.show()

print("\n✅ Figure 9 generated: Comprehensive Summary Dashboard (updated with k=10)")

## 13. Summary Table for Thesis

In [ ]:
# Create publication-ready summary table
summary_table = df_summary[['scenario', 'precision@5_mean', 'precision@5_std',
                             'recall@5_mean', 'recall@5_std',
                             'f1@5_mean', 'f1@5_std',
                             'map_mean', 'map_std',
                             'mrr_mean', 'mrr_std',
                             'ndcg@5_mean', 'ndcg@5_std',
                             'time_mean_ms', 'time_std_ms']].copy()

# Rename columns for readability
summary_table['scenario'] = summary_table['scenario'].map(SHORT_LABELS)
summary_table.columns = ['Method', 'P@5', 'P@5_std', 'R@5', 'R@5_std', 
                         'F1@5', 'F1@5_std', 'MAP', 'MAP_std',
                         'MRR', 'MRR_std', 'NDCG@5', 'NDCG@5_std',
                         'Latency(ms)', 'Lat_std']

# Format for display
display_table = summary_table.copy()
for col in display_table.columns[1:]:
    if 'std' in col:
        display_table[col] = display_table[col].apply(lambda x: f'±{x:.3f}')
    elif 'Latency' in col:
        display_table[col] = display_table[col].apply(lambda x: f'{x:.1f}')
    else:
        display_table[col] = display_table[col].apply(lambda x: f'{x:.3f}')

# Combine mean and std
result_table = pd.DataFrame({
    'Method': display_table['Method'],
    'Precision@5': display_table['P@5'] + ' ' + display_table['P@5_std'],
    'Recall@5': display_table['R@5'] + ' ' + display_table['R@5_std'],
    'F1@5': display_table['F1@5'] + ' ' + display_table['F1@5_std'],
    'MAP': display_table['MAP'] + ' ' + display_table['MAP_std'],
    'MRR': display_table['MRR'] + ' ' + display_table['MRR_std'],
    'NDCG@5': display_table['NDCG@5'] + ' ' + display_table['NDCG@5_std'],
    'Latency(ms)': display_table['Latency(ms)'] + ' ' + display_table['Lat_std']
})

print("\n📊 Summary Table for Thesis (Mean ± Std)\n")
print(result_table.to_string(index=False))

# Save as CSV and LaTeX
result_table.to_csv(OUTPUT_DIR / 'table1_summary_results.csv', index=False)
result_table.to_latex(OUTPUT_DIR / 'table1_summary_results.tex', index=False, escape=False)

print("\n✅ Summary table saved as CSV and LaTeX")

## 14. Key Findings Summary

In [ ]:
# Calculate key findings
print("\n" + "="*60)
print("KEY FINDINGS FROM EXPERIMENT 1")
print("="*60 + "\n")

# Best performers
best_map = df_summary.loc[df_summary['map_mean'].idxmax()]
print(f"🏆 BEST OVERALL (MAP): {SHORT_LABELS[best_map['scenario']]}")
print(f"   MAP = {best_map['map_mean']:.4f} ± {best_map['map_std']:.4f}")
print()

# Compare to baseline
baseline = df_summary[df_summary['scenario'] == 'BM25_Baseline'].iloc[0]
best_improvement = ((best_map['map_mean'] - baseline['map_mean']) / baseline['map_mean']) * 100
print(f"📈 IMPROVEMENT OVER BM25 BASELINE: {best_improvement:.1f}%")
print()

# Precision leader
best_precision = df_summary.loc[df_summary['precision@5_mean'].idxmax()]
print(f"🎯 HIGHEST PRECISION@5: {SHORT_LABELS[best_precision['scenario']]}")
print(f"   P@5 = {best_precision['precision@5_mean']:.4f}")
print()

# Recall leader
best_recall = df_summary.loc[df_summary['recall@5_mean'].idxmax()]
print(f"🔍 HIGHEST RECALL@5: {SHORT_LABELS[best_recall['scenario']]}")
print(f"   R@5 = {best_recall['recall@5_mean']:.4f}")
print()

# Fastest
fastest = df_summary.loc[df_summary['time_mean_ms'].idxmin()]
print(f"⚡ FASTEST: {SHORT_LABELS[fastest['scenario']]}")
print(f"   Latency = {fastest['time_mean_ms']:.1f} ms")
print()

# Statistical significance summary
sig_comparisons = df_stats[df_stats['bonferroni_significant'] == True]
print(f"📊 STATISTICAL ANALYSIS:")
print(f"   Total pairwise comparisons: {len(df_stats)}")
print(f"   Statistically significant: {len(sig_comparisons)}")
print(f"   Significance rate: {len(sig_comparisons)/len(df_stats)*100:.1f}%")
print()

print("="*60)

# Save findings to text file
with open(OUTPUT_DIR / 'key_findings.txt', 'w') as f:
    f.write("KEY FINDINGS FROM EXPERIMENT 1\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Best Overall (MAP): {SHORT_LABELS[best_map['scenario']]}\n")
    f.write(f"MAP = {best_map['map_mean']:.4f} ± {best_map['map_std']:.4f}\n\n")
    f.write(f"Improvement over BM25: {best_improvement:.1f}%\n\n")
    f.write(f"Highest Precision@5: {SHORT_LABELS[best_precision['scenario']]} = {best_precision['precision@5_mean']:.4f}\n")
    f.write(f"Highest Recall@5: {SHORT_LABELS[best_recall['scenario']]} = {best_recall['recall@5_mean']:.4f}\n")
    f.write(f"Fastest: {SHORT_LABELS[fastest['scenario']]} = {fastest['time_mean_ms']:.1f} ms\n")

print("\n✅ Key findings saved to key_findings.txt")

## 15. Export All Results

In [ ]:
print("\n" + "="*60)
print("EXPORT SUMMARY")
print("="*60 + "\n")

print(f"📁 Output Directory: {OUTPUT_DIR}\n")
print("Generated Files:")
print("  Figures (9 total):")
print("    ✓ fig1_overall_performance.png/.pdf")
print("    ✓ fig2_precision_at_k.png/.pdf")
print("    ✓ fig3_recall_at_k.png/.pdf")
print("    ✓ fig4_f1_scores.png/.pdf")
print("    ✓ fig5_precision_recall_tradeoff.png/.pdf")
print("    ✓ fig6_ndcg_progression.png/.pdf")
print("    ✓ fig7_latency_analysis.png/.pdf")
print("    ✓ fig8_statistical_significance.png/.pdf")
print("    ✓ fig9_comprehensive_dashboard.png/.pdf")
print("\n  Tables:")
print("    ✓ table1_summary_results.csv")
print("    ✓ table1_summary_results.tex")
print("\n  Analysis:")
print("    ✓ key_findings.txt")
print("\n" + "="*60)
print("\n✅ All visualizations and tables generated successfully!")
print("\n📝 Ready for thesis inclusion")
print(f"\n🎓 Use these figures in your thesis Chapter 4 (Experimental Evaluation)")

## Notes for Thesis

### Recommended Figure Placement

1. **Section 4.1.4.1 - Overall Performance**
   - Figure 1: Overall Performance Comparison (MAP, MRR, NDCG@5)
   - Table 1: Summary Results

2. **Section 4.1.4.2 - Precision & Recall Analysis**
   - Figure 2: Precision@k Comparison
   - Figure 3: Recall@k Comparison
   - Figure 5: Precision-Recall Tradeoff

3. **Section 4.1.4.3 - Ranking Quality**
   - Figure 6: NDCG@k Progression
   - Figure 4: F1@k Scores

4. **Section 4.1.4.4 - Computational Efficiency**
   - Figure 7: Latency Analysis

5. **Section 4.1.4.5 - Statistical Validation**
   - Figure 8: Statistical Significance Heatmap

6. **Appendix or Summary Section**
   - Figure 9: Comprehensive Dashboard

### Key Points to Highlight

1. Dense BGE-M3 achieves highest MAP (0.546) - 108% improvement over BM25
2. SPLADE significantly outperforms traditional BM25 (79% improvement)
3. Hybrid methods show varying effectiveness depending on component combination
4. All improvements are statistically significant (p < 0.001, large effect sizes)
5. Trade-off between latency and quality exists (dense methods are slower)

### LaTeX Integration

```latex
\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.9\textwidth]{figures/fig1_overall_performance.pdf}
    \caption{Overall retrieval quality comparison across five methods. Dense BGE-M3 achieves 
    the highest performance across all metrics (MAP=0.546, MRR=0.928, NDCG@5=0.744), 
    representing a 108\% improvement over the BM25 baseline. Error bars show ±1 standard deviation.}
    \label{fig:exp1_overall}
\end{figure}
```